## Problem Set 4: Nondeterministic FAs
_csc427, semester 222
<br>
university of miami
<br>
date: 16 feb 2022_


---

### Student name: Dominick Bello 

---

### Discussion


The NondeterministicFiniteAutomata class runs an NFA. We presented three perspectives on non-determinism,

- The Oracle. There are choices in the computation. We consult an oracle with each choice and it tells us which choice to make. If the string is in the language then the oracle gives us a series of choices which lead to an accepting state. If the string is not in the language then the oracle just answers randomly.
- There Exists. For a string in the language we are presented with a computation path on that string that leads to an accepting state. We do not wonder about how this computation path was found.
- Search. We try all computation paths, thus turning the machine into a deterministic machine.

We have to use the third strategy because we are all out of Oracles and other magical beasts.


__How to search:__

There are certainly numerous ways of search. It must be that,

- The search will certainly halt with an accept or non-accept answer (that is correct!);
- and that the serach will halt soon. A predictable time limit based on the string length.

Our machine is based on running all computation routes in parallel and keeping a set of possibilities of states having looked at the input so far. With each next symbol, each of the possible states is considered, as to the set of possibilities for that state, symbol combination, and these are unioned up into the set of possible states now, after having looked at that one more symbol. 

Note that: in our convention it is possible that the either the set of possibilities is empty, or the state, symbol combination yeilds an empty set, or that there is no key on our transition map for a state, symbol combination.

- If the set of possibilities is empty, the machine will continue reading input until the end, and never again have possible states. This string will not be accepted.
- If the there is no key in the dictionary, that is treated as having a key yielding the empty set. This is a convenience. 

__Epsilon Closures__

The search must consider all possible epsilon moves. This tricky problem is handled by an epsilon closure operator.

A set of states is epsilon closed if all paths by epsilon transitions from a state in the set yeilds a state in the state. Another way said, you can't find an epsilon move to escape an epsilon closed set. We make sure
that at all moments that count, our set of possibilities are epsilon closed sets.

It is called a closure operator because,

- Applying it to a set yeilds a (possibily non-strict) superset.
- Applying it twice yeilds the set unchanged.


__Machine Description__

The machine description for an NFA differs in three ways from the description of a simple FA (a DFA),

- The NFA transition function yields a set of states, instead of a single state when a DFA.
- There is a special symbol epsilon (written as :) that can be used as a symbol in the state transition function.
- It is allowed that a "key is missing" from the transition dictionary, in the case of no outgoing arrow from that state with that symbol.

Note that the special letter : is not included in the alphabet or allowed in a the string, but it is allowed as a symbol in the transition map.





### Exercise A

Finish the code for the NondeterministicFiniteAutomata class.


In [1]:
class NondeterministicFiniteAutomata:
   
    def __init__(self,fa_description):
        self.fa = fa_description
        # the : is reserved
        assert ':' not in self.fa['alphabet']
        self.states = self.epsilon_close({self.fa['start']})
    

    def epsilon_one_step(self,state_set):
        epsilon = set()
        for state in state_set:
            target = self.fa['transitions'].get((state, ':'), set())
            epsilon = epsilon.union(target)

        return epsilon 
    
    def epsilon_close(self,state_set):
        close = set()
        while close != state_set:
            close = state_set
            state_set = close.union(self.epsilon_one_step(close))
        
        return close 

    def one_step(self,symbol):
        assert symbol in self.fa['alphabet']
        target = set()
        for state in self.states:
            oneStepTarget = self.fa['transitions'].get((state, symbol),
            set())
            target = target.union(oneStepTarget)
        self.states = self.epsilon_close(target)
        return self.states

    
        return self.states 
            
    def compute(self,string,verbose=False):
        for symbol in string:
            if verbose:
                print(f'{self.states} --{symbol}--> {self.one_step(symbol)}')
            else:
                self.one_step(symbol)
        accept = len(self.states.intersection(self.fa['accept'])) != 0
        self.states = self.epsilon_close({self.fa['start']})

        return accept


# end class NondeterministicFiniteAutomata

class NFA_Test:
    
    def __init__(self, fa_description, fa_name):
        self.nfa = NondeterministicFiniteAutomata(fa_description)
        self.fa_name = fa_name
        
    def test(self, test_vect, verbose=False):
        tv_true, tv_false = test_vect
        correct = 0 

        print(f'*** testing {self.fa_name}')
        for string in tv_true:
            if self.nfa.compute(string):
                correct += 1
            else:
                print(f'should accept but does not: |{string}| ')
        print(f'\t{correct} correctly accepted out of {len(tv_true)} strings')
        passed = correct == len(tv_true)

        correct = 0 
        for string in tv_false:
            if not self.nfa.compute(string):
                correct += 1
            else:
                print(f'should reject but does not: |{string}| ')
        print(f'\t{correct} correctly rejected out of {len(tv_false)} strings')
 
        passed = passed and (correct == len(tv_false))
        if passed:
            print(f'*** PASSES\n')
        else:
            print(f'*** FAILS\n')
        return passed

# end class NFA_Test


def nfa_test_helper(nfa_bundle):

    correct = 0 
    for nfa in nfa_bundle:
        desc ,test, name = nfa
        nfa_t = NFA_Test(desc,name)
        if nfa_t.test(test):
            correct += 1
    return correct == len(nfa_bundle)


### Exercise B

Examples of NFA's from the body of the text of the class textbook. 

Transcribe the NFA diagrams into the code descriptions, and test.


In [2]:

# accepts all strings that contain either 101 or 11 as a substring.

Sipser_N1 = {
    'states':set({'q1','q2','q3','q4'}),
    'alphabet':set({'0','1'}),
    'transitions':{
        ('q1','0') : set({'q1'}),
        ('q1','1') : set({'q1','q2'}),
        ('q2','0') : set({'q3'}),
        ('q2',':') : set({'q3'}),
        ('q3','1') : set({'q4'}),
        ('q4','0') : set({'q4'}),
        ('q4','1') : set({'q4'})
    },
    'start':'q1',
    'accept':set({'q4'})

}

Sipser_N1_test = (['010110','101','11'],['0','10','00100'])


# accepts all strings containing a 1 in the third position from the end

Sipser_N2 = {
    'states':set({'q1','q2','q3','q4'}),
    'alphabet':set({'0','1'}),
    'transitions':{
        ('q1','0') : set({'q1'}),
        ('q1','1') : set({'q1','q2'}),
        ('q2','0') : set({'q3'}),
        ('q2','1') : set({'q3'}),
        ('q3','0') : set({'q4'}),
        ('q3','1') : set({'q4'})
    },
    'start':'q1',
    'accept':set({'q4'})

}

Sipser_N1_test = (['010110','101','11'],['0','10','00100'])


Sipser_N2_test = (['100','1111','00100'],['0','011','1011'])


# accepts all strings of 0's of length k where k is a multiple of 2 or 3

Sipser_N3 = {
    'states':set({'q1','q2','q3','q4','q5','q6'}),
    'alphabet':set({'0'}),
    'transitions':{
        ('q1',':') : set({'q2','q3'}),
        ('q2','0') : set({'q4'}),
        ('q4','0') : set({'q2'}),
        ('q3','0') : set({'q5'}),
        ('q5','0') : set({'q6'}),
        ('q6','0') : set({'q3'})
    },
    'start':'q1',
    'accept':set({'q2','q3'})
}


Sipser_N3_test = (['000','0000','000000'],['00000','0000000','00000000000'])


# try to describe this, better yet, write an RE fro it

Sipser_N4 = {
      'states':set({'q1','q2','q3'}),
    'alphabet':set({'a','b'}),
    'transitions':{
        ('q1','b') : set({'q2'}),
        ('q1',':') : set({'q3'}),
        ('q2','a') : set({'q2','q3'}),
        ('q2','b') : set({'q3'}),
        ('q3','a') : set({'q1'})
    },
    'start':'q1',
    'accept':set({'q1'})

}

Sipser_N4_test = (['a','baba','baa'],['b','bb','babba'])



nfas= [
    (Sipser_N1,Sipser_N1_test,'Sipser N1'),
    (Sipser_N2,Sipser_N2_test,'Sipser N2'),
    (Sipser_N3,Sipser_N3_test,'Sipser N3'), 
    (Sipser_N4,Sipser_N4_test,'Sipser N4')
]

nfa_test_helper(nfas)

*** testing Sipser N1
	3 correctly accepted out of 3 strings
	3 correctly rejected out of 3 strings
*** PASSES

*** testing Sipser N2
	3 correctly accepted out of 3 strings
	3 correctly rejected out of 3 strings
*** PASSES

*** testing Sipser N3
	3 correctly accepted out of 3 strings
	3 correctly rejected out of 3 strings
*** PASSES

*** testing Sipser N4
	3 correctly accepted out of 3 strings
	3 correctly rejected out of 3 strings
*** PASSES



True

### Exercise C

Give the NFA's for the following languages from the textbook, and test.

- Exercise 1.7 b, c, and e.
- Exercise 1.20 b.
- Exercise 1.17 a.
- Exercise 1.19 a, b, c.


In [3]:

# 1.7b The language "contains the substring 0101" with five states

ex_1_7b = {
      'states':set({'q1','q2','q3','q4','q5'}),
    'alphabet':set({'0','1'}),
    'transitions':{
        ('q1','0') : set({'q1','q2'}),
        ('q1','1') : set({'q1'}),
        ('q2','1') : set({'q3'}),
        ('q3','0') : set({'q4'}),
        ('q4','1') : set({'q5'}),
        ('q5','0') : set({'q5'}),
        ('q5','1') : set({'q5'})
    },
    'start':'q1',
    'accept':set({'q5'})

}
ex_1_7b_test = (['0101','00101','000101110'],
                ['010','0100','0110'])


# 1.7c The language " contains an even number of 0s, or contains exactly two 1s" with six states

ex_1_7c = {
   'states':set({'q1','q2','q3','q4','q5','q6'}),
    'alphabet':set({'0','1'}),
    'transitions':{
        ('q1',':') : set({'q2','q3'}),
        ('q2','1') : set({'q2'}),
        ('q2','0') : set({'q4'}),
        ('q4','0') : set({'q2'}),
        ('q4','1') : set({'q4'}),
        ('q3','0') : set({'q3'}),
        ('q3','1') : set({'q5'}),
        ('q5','0') : set({'q5'}),
        ('q5','1') : set({'q6'}),
        ('q6','0') : set({'q6'})
    },
    'start':'q1',
    'accept':set({'q2','q6'})


}
ex_1_7c_test = (['11','011','010'],
                ['01','110100'])

# 1.7e The language 0*1*0+ with three states

ex_1_7e = {
    'states':set({'q1','q2','q3'}),
    'alphabet':set({'0','1'}),
    'transitions':{
        ('q1','0') : set({'q1'}),
        ('q1',':') : set({'q2'}),
        ('q2','1') : set({'q2'}),
        ('q2','0') : set({'q3'}),
        ('q3','0') : set({'q3'})
    },
    'start':'q1',
    'accept':set({'q3'}) 

}
ex_1_7e_test = (['10','110','011000'],
               ['1','001','01010'])


# 1.20(b) a(ba)*b

ex_1_20b = {
   'states':set({'q1','q2','q3','q4'}),
    'alphabet':set({'a','b'}),
    'transitions':{
        ('q1','a') : set({'q2'}),
        ('q2','b') : set({'q3'}),
        ('q3','a') : set({'q4'}),
        ('q4','b') : set({'q3'})
    },
    'start':'q1',
    'accept':set({'q3'}) 

}
ex_1_20b_test = (['ab','abab','ababab'],
                 ['a','b','abaab'])



# 1.17a (01 U 001 U 010)*

ex_1_17a = {
    'states':set({'q1','q2','q3','q4','q5','q6','q7','q8'}),
    'alphabet':set({'0','1'}),
    'transitions':{
        ('q1','0') : set({'q2'}),
        ('q2','0') : set({'q3'}),
        ('q2','1') : set({'q4'}),
        ('q3','1') : set({'q5'}),
        ('q4','0') : set({'q6'}),
        ('q5','0') : set({'q2'}),
        ('q6','0') : set({'q7'}),
        ('q6','1') : set({'q4'}),
        ('q7','0') : set({'q3'}),
        ('q7','1') : set({'q8'}),
        ('q8','0') : set({'q6'})
    },
    'start':'q1',
    'accept':set({'q1','q4','q5','q6','q8'})

}

ex_1_17a_test = (['010','01001','01001'],
                ['011','000','0110'])


# 1.19(a) (0 U 1)* 000(0 U 1)*

ex_1_19a = {
   'states':set({'q1','q2','q3','q4','q5','q6'}),
    'alphabet':set({'0','1'}),
    'transitions':{
        ('q1','0') : set({'q1'}),
        ('q1','1') : set({'q1'}),
        ('q1',':') : set({'q2'}),
        ('q2','0') : set({'q3'}),
        ('q3','0') : set({'q4'}),
        ('q4','0') : set({'q5'}),
        ('q5',':') : set({'q6'}),
        ('q6','0') : set({'q6'}),
        ('q6','1') : set({'q6'}),
    },
    'start':'q1',
    'accept':set({'q6'})

}
ex_1_19a_test = (['10001','0001','11000'],
                ['00','1','1001001',])


# 1.19(b) ( ((00)*(11)) U 01 )*   

ex_1_19b = {
   'states':set({'q1','q2','q3','q4','q5','q6','q7','q8','q9','q10','q11','q12','q13','q14'}),
    'alphabet':set({'0','1'}),
    'transitions':{
        ('q1',':') : set({'q2','q14'}),
        ('q2',':') : set({'q3','q10'}),
        ('q3',':') : set({'q4','q7'}),
        ('q4','0') : set({'q5'}),
        ('q5','0') : set({'q6'}),
        ('q6',':') : set({'q4','q7'}),
        ('q7','1') : set({'q8'}),
        ('q8','1') : set({'q9'}),
        ('q9',':') : set({'q13'}),
        ('q10','0') : set({'q11'}),
        ('q11','1') : set({'q12'}),
        ('q12',':') : set({'q13'}),
        ('q13',':') : set({'q2','q14'})
    },
    'start':'q1',
    'accept':set({'q14'})

}

ex_1_19b_test = (['0011','0111','010011'],
                 ['001','011','0001'])



# 1.19(c) empty-set*

ex_1_19c = {
   'states':set({'q1','q2','q3'}),
    'alphabet':set({'0','1'}),
    'transitions':{
        ('q1',':') : set({'q2'}),
        ('q1','0') : set({'q3'}),
        ('q1','1') : set({'q3'}),
        ('q3','0') : set({'q3'}),
        ('q3','1') : set({'q3'})
    },
    'start':'q1',
    'accept':set({'q2'})

}
ex_1_19c_test = ([''],['0','1','00']) 




In [10]:

nfas= [
    (ex_1_7b,ex_1_7b_test,'Ex 1.7 (b)'),
    (ex_1_7c,ex_1_7c_test,'Ex 1.7 (c)'),
    (ex_1_7e,ex_1_7e_test,'Ex 1.7 (e)'),
    (ex_1_20b,ex_1_20b_test,'Ex 1.20 (b)'),
    (ex_1_17a,ex_1_17a_test,'Ex 1.17 (a)'),
    (ex_1_19a,ex_1_19a_test,'Ex 1.19 (a)'),
    (ex_1_19b,ex_1_19b_test,'Ex 1.19 (b)'),
    (ex_1_19c,ex_1_19c_test,'Ex 1.19 (c)')
]

nfa_test_helper(nfas)

*** testing Ex 1.7 (b)
	3 correctly accepted out of 3 strings
	3 correctly rejected out of 3 strings
*** PASSES

*** testing Ex 1.7 (c)
	3 correctly accepted out of 3 strings
	2 correctly rejected out of 2 strings
*** PASSES

*** testing Ex 1.7 (e)
	3 correctly accepted out of 3 strings
	3 correctly rejected out of 3 strings
*** PASSES

*** testing Ex 1.20 (b)
	3 correctly accepted out of 3 strings
	3 correctly rejected out of 3 strings
*** PASSES

*** testing Ex 1.17 (a)
	3 correctly accepted out of 3 strings
	3 correctly rejected out of 3 strings
*** PASSES

*** testing Ex 1.19 (a)
	3 correctly accepted out of 3 strings
	3 correctly rejected out of 3 strings
*** PASSES

*** testing Ex 1.19 (b)
	3 correctly accepted out of 3 strings
	3 correctly rejected out of 3 strings
*** PASSES

*** testing Ex 1.19 (c)
	1 correctly accepted out of 1 strings
	3 correctly rejected out of 3 strings
*** PASSES



True

### End of assigment
